In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [2]:
elec = pd.read_csv("electronics.csv")
mod = pd.read_csv("modcloth.csv")

# **Recommendaion System**

### **Modcloth**

In [3]:
mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99893 entries, 0 to 99892
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_id     99893 non-null  int64  
 1   user_id     99892 non-null  object 
 2   rating      99893 non-null  int64  
 3   timestamp   99893 non-null  object 
 4   size        78133 non-null  float64
 5   fit         81387 non-null  object 
 6   user_attr   91526 non-null  object 
 7   model_attr  99893 non-null  object 
 8   category    99893 non-null  object 
 9   brand       25913 non-null  object 
 10  year        99893 non-null  int64  
 11  split       99893 non-null  int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 9.1+ MB


In [4]:
mod=mod.drop(columns=['brand'],axis=1)
mod.dropna(subset=['user_id'],inplace=True)

In [5]:
mod.item_id.nunique()

1020

In [6]:
mod.user_id.nunique()

44783

In [7]:
new_df_mod=mod.groupby("item_id").filter(lambda x:x['rating'].count() >=20)
new_df_mod.head()

,item_id,user_id,rating,timestamp,size,fit,user_attr,model_attr,category,year,split
0,7443,Alex,4,2010-01-21 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,2012,0
1,7443,carolyn.agan,3,2010-01-27 08:00:00+00:00,NaN,NaN,NaN,Small,Dresses,2012,0
2,7443,Robyn,4,2010-01-29 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,2012,0
3,7443,De,4,2010-02-13 08:00:00+00:00,NaN,NaN,NaN,Small,Dresses,2012,0
4,7443,tasha,4,2010-02-18 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,2012,0


In [8]:
ratings_matrix_mod = new_df_mod.pivot_table(values='rating', index='user_id', columns='item_id', fill_value=0)
ratings_matrix_mod.head()

item_id,6454,7443,11960,16411,21296,22563,24853,27439,27590,28252,...,154540,154541,154543,154567,154661,154665,154794,154797,155293,155317
user_id,,,,,,,,,,,,,,,,,,,,,
"""Ferrari"")",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#1dad,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
'Chelle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
'Tree',0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X_mod = ratings_matrix_mod.T
X_mod.head()

user_id,"""Ferrari"")",#,#1dad,'Chelle,'Tree',(usually),-L,.,..,00izzyb,...,zuel,zugai01,zulemaphone,zumbafitnesscarly,zumbaneko,zurajohnson,zuzu_zoom,🇦🇺,🐻,😊
item_id,,,,,,,,,,,,,,,,,,,,,
6454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16411,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21296,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
SVD_mod = TruncatedSVD(n_components=10)
decomposed_matrix_mod = SVD_mod.fit_transform(X_mod)
decomposed_matrix_mod.shape

(482, 10)

In [11]:
#Finding Correlation between users
correlation_matrix_mod = np.corrcoef(decomposed_matrix_mod)
correlation_matrix_mod.shape

(482, 482)

In [12]:
def recommend_items_mod(item):
    item_ids = list(X_mod.index)
    item_id = item_ids.index(item)
    corr_item_id = correlation_matrix_mod[item_id]
    recommend = list(X_mod.index[corr_item_id > 0.65])
    recommend.remove(item)
    return recommend[0:20]

In [13]:
recommend_items_mod(7443)

[6454,
 11960,
 16411,
 22563,
 24853,
 27439,
 27590,
 28252,
 28967,
 31644,
 31752,
 32134,
 32236,
 34931,
 34937,
 35272,
 35525,
 36172,
 36408,
 40141]

# Electronics Dataset

In [15]:
elec.head()

,item_id,user_id,rating,timestamp,model_attr,category,brand,year,user_attr,split
0,0,0,5.0,1999-06-13,Female,Portable Audio & Video,NaN,1999,NaN,0
1,0,1,5.0,1999-06-14,Female,Portable Audio & Video,NaN,1999,NaN,0
2,0,2,3.0,1999-06-17,Female,Portable Audio & Video,NaN,1999,NaN,0
3,0,3,1.0,1999-07-01,Female,Portable Audio & Video,NaN,1999,NaN,0
4,0,4,2.0,1999-07-06,Female,Portable Audio & Video,NaN,1999,NaN,0


In [16]:
elec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292954 entries, 0 to 1292953
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   item_id     1292954 non-null  int64  
 1   user_id     1292954 non-null  int64  
 2   rating      1292954 non-null  float64
 3   timestamp   1292954 non-null  object 
 4   model_attr  1292954 non-null  object 
 5   category    1292954 non-null  object 
 6   brand       331120 non-null   object 
 7   year        1292954 non-null  int64  
 8   user_attr   174124 non-null   object 
 9   split       1292954 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 98.6+ MB


In [17]:
#we dont need users who haven't buyed multiple products
a=elec.user_id.value_counts()
a

142967    41
30661     38
89185     37
80476     34
46878     34
          ..
354760     1
352713     1
356811     1
346572     1
2047       1
Name: user_id, Length: 1157633, dtype: int64

In [18]:
indexes = []
for i in a.index:
    if a[i] > 2:
        indexes.append(i)
len(indexes)

20335

In [19]:
elec2 = elec[elec.user_id.isin(indexes)]

In [20]:
elec2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73181 entries, 7 to 1292855
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_id     73181 non-null  int64  
 1   user_id     73181 non-null  int64  
 2   rating      73181 non-null  float64
 3   timestamp   73181 non-null  object 
 4   model_attr  73181 non-null  object 
 5   category    73181 non-null  object 
 6   brand       19493 non-null  object 
 7   year        73181 non-null  int64  
 8   user_attr   30507 non-null  object 
 9   split       73181 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 6.1+ MB


In [21]:
new_df_elec=elec2.groupby("item_id").filter(lambda x:x['rating'].count() >=20)

In [22]:
ratings_matrix_elec = new_df_elec.pivot_table(values='rating', index='user_id', columns='item_id', fill_value=0)
ratings_matrix_elec.head()

item_id,2,17,43,77,79,80,81,87,111,121,...,9175,9250,9255,9297,9315,9345,9361,9406,9510,9537
user_id,,,,,,,,,,,,,,,,,,,,,
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
ratings_matrix_elec.shape

(19394, 801)

In [24]:
#as the matrix is sparse, apply transpose
X_elec = ratings_matrix_elec.T
X_elec.head()

user_id,7,28,62,96,131,158,269,372,392,428,...,1140484,1141428,1142408,1144602,1144732,1144754,1148742,1149153,1150395,1153360
item_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X_elec.shape

(801, 19394)

In [26]:
SVD_elec = TruncatedSVD(n_components=10)
decomposed_matrix_elec = SVD_elec.fit_transform(X_elec)
decomposed_matrix_elec.shape

(801, 10)

In [27]:
correlation_matrix_elec = np.corrcoef(decomposed_matrix_elec)
correlation_matrix_elec.shape

(801, 801)

In [28]:
correlation_matrix_elec

array([[ 1.00000000e+00,  2.12692511e-01, -3.88264377e-04, ...,
         4.16338776e-01,  8.10307780e-01,  3.94393504e-01],
       [ 2.12692511e-01,  1.00000000e+00,  7.68884513e-01, ...,
         2.13361466e-01,  2.67251083e-01,  3.47547300e-01],
       [-3.88264377e-04,  7.68884513e-01,  1.00000000e+00, ...,
         3.62436360e-01,  1.68353019e-01,  5.38589284e-01],
       ...,
       [ 4.16338776e-01,  2.13361466e-01,  3.62436360e-01, ...,
         1.00000000e+00,  3.85906189e-01,  7.28804654e-01],
       [ 8.10307780e-01,  2.67251083e-01,  1.68353019e-01, ...,
         3.85906189e-01,  1.00000000e+00,  3.48407960e-01],
       [ 3.94393504e-01,  3.47547300e-01,  5.38589284e-01, ...,
         7.28804654e-01,  3.48407960e-01,  1.00000000e+00]])

In [29]:
def recommend_items_elec(item):
    item_ids = list(X_elec.index)
    item_id = item_ids.index(item)
    corr_item_id = correlation_matrix_elec[item_id]
    recommend = list(X_elec.index[corr_item_id > 0.65])
    recommend.remove(item)
    return recommend[0:20]

In [30]:
recommend_items_elec(2)

[77,
 79,
 111,
 220,
 259,
 296,
 511,
 619,
 647,
 714,
 836,
 850,
 1105,
 1111,
 1166,
 1258,
 1294,
 1297,
 1372,
 1428]

# GUI

In [32]:
import tkinter as tk
from tkinter import *

In [44]:
def gui_mod():
    f = tk.Tk()
    f.title("Recommend clothing items")
    f.geometry('600x400')
    lbl = tk.Label(f, text = "Enter Item id:")
    lbl.pack()
    def rec():
        item = int(txt.get(1.0, "end-1c"))
        items=recommend_items_mod(item)
        for i in range(0,len(items)):
            listbox.insert(i,items[i])
    txt=tk.Text(f,height = 3,width = 7)
    txt.pack()
    btn = tk.Button(f,text = "Submit", command = rec)
    btn.pack()
    lblop = tk.Label(f, text = "The recommended clothing items are:")
    lblop.pack()
    listbox = Listbox(f, height = 22, width = 15, activestyle = 'dotbox', font = "Helvetica",fg = "red")
    listbox.pack()
    f.mainloop()

In [43]:
def gui_elec():
    f = tk.Tk()
    f.title("Recommend electronic items")
    f.geometry('600x400')
    lbl = tk.Label(f, text = "Enter Item id:")
    lbl.pack()
    def rec():
        item = int(txt.get(1.0, "end-1c"))
        items=recommend_items_elec(item)
        for i in range(0,len(items)):
            listbox.insert(i,items[i])
    txt=tk.Text(f,height = 3,width = 7)
    txt.pack()
    btn = tk.Button(f,text = "Submit", command = rec)
    btn.pack()
    lblop = tk.Label(f, text = "The recommended electronics items are:")
    lblop.pack()
    listbox = Listbox(f, height = 22, width = 15, activestyle = 'dotbox', font = "Helvetica",fg = "red")
    listbox.pack()
    f.mainloop()

In [45]:
gui_mod()

In [46]:
gui_elec()